## 🧠 1. CONCEPT: What is Pathfinding?

Pathfinding in Roblox is like giving NPCs a brain GPS 🛰️.
Using the PathfindingService, they calculate the safest, shortest route to reach a target — while avoiding lava pits, jumping over obstacles, or even climbing walls.

## 🧭 2. CALCULATING PATHS – CreatePath() & ComputeAsync()

✅ Step 1: Create the path

In [ ]:
local pathfindingService = game:GetService("PathfindingService")

local path = pathfindingService:CreatePath({
	AgentRadius = 2,
	AgentHeight = 5,
	AgentCanJump = true,
	AgentJumpHeight = 10,
	AgentCanClimb = true,
	AgentMaxSlope = 45
})

✅ Step 2: Compute the path to destination

In [ ]:
path:ComputeAsync(npc.Position, target.Position)

✅ Step 3: Check success and get waypoints

In [ ]:
if path.Status == Enum.PathStatus.Success then
	local waypoints = path:GetWaypoints()
end

## 🚶‍♂️ 3. MOVE NPC ALONG PATH (+ JUMPING & CLIMBING)

🧠 Full Moving Script Example:

In [ ]:
local npc = script.Parent
local humanoid = npc:WaitForChild("Humanoid")
local pathfindingService = game:GetService("PathfindingService")
local target = workspace.TargetPart

local path = pathfindingService:CreatePath()
path:ComputeAsync(npc.Position, target.Position)

if path.Status == Enum.PathStatus.Success then
	local waypoints = path:GetWaypoints()
	for _, waypoint in ipairs(waypoints) do
		if waypoint.Action == Enum.PathWaypointAction.Jump then
			humanoid:ChangeState(Enum.HumanoidStateType.Jumping)
		end
		humanoid:MoveTo(waypoint.Position)
		humanoid.MoveToFinished:Wait()
	end
else
	warn("❌ Path not found")
end


## 🧩 4. MODIFIER #1 – Setting Material Costs

In [ ]:
local path = pathfindingService:CreatePath({
	Costs = {
		Water = math.huge,  -- Avoid completely
		Plastic = 10,        -- Less preferred
		Grass = 1            -- Preferred
	}
})

## 🚧 5. MODIFIER #2 – Defining Unwalkable Regions

Make regions where the NPC should never go.

    🛑 Use PathfindingModifier:

    Add PathfindingModifier object to any part in the map.

    Set PassThrough = false
    This marks that area as off-limits for pathfinding.

## 🎯 6. MODIFIER #3 – Choosing Specific Paths (Preferred routes)

Use PassThrough = true and lower material cost to encourage specific path directions.

Example: Make “Safe Path” parts that the NPC will prefer.

## ⚡ 7. PRO TIPS for Pathfinding Ninjas

| 🧠 Tip                       | 💡 Why it’s awesome                               |
| ---------------------------- | ------------------------------------------------- |
| Cache the last computed path | Save performance and prevent constant recompute   |
| Use raycasting with pathing  | Avoid edge cases like flying/falling off          |
| Recalculate on fail/detour   | Helps if player moves or something blocks the way |
| Keep target on ground        | Avoid computing paths to unreachable high points  |


## 🧠 8. ADVANCED TECHNIQUES 💎

✅ Dynamic Re-pathing (every few seconds):

✅ Avoiding Other NPCs:

Use invisible collision parts (PathfindingModifiers) dynamically placed where NPCs stand.

In [ ]:
while true do
	task.wait(2)
	path:ComputeAsync(npc.Position, player.Position)
end

## 🧍‍♂️ 9. NPC FOLLOWING PLAYER – COMPLETE SCRIPT

📍 ServerScript inside NPC model:

In [ ]:
local npc = script.Parent
local humanoid = npc:WaitForChild("Humanoid")
local pathfindingService = game:GetService("PathfindingService")
local runService = game:GetService("RunService")
local targetPlayer = game.Players:GetPlayers()[1] -- Follows first player

function followPlayer()
	while targetPlayer and targetPlayer.Character do
		local targetHRP = targetPlayer.Character:WaitForChild("HumanoidRootPart")

		local path = pathfindingService:CreatePath()
		path:ComputeAsync(npc.Position, targetHRP.Position)

		if path.Status == Enum.PathStatus.Success then
			for _, waypoint in pairs(path:GetWaypoints()) do
				if waypoint.Action == Enum.PathWaypointAction.Jump then
					humanoid:ChangeState(Enum.HumanoidStateType.Jumping)
				end
				humanoid:MoveTo(waypoint.Position)
				humanoid.MoveToFinished:Wait()
			end
		end

		task.wait(1) -- Re-path every second
	end
end

followPlayer()


## 🔥 FULL NPC FOLLOWING + DESPAWN + RANGE LOGIC SCRIPT:

    👀 Detect players nearby (50 studs)

    🏃‍♂️ Chase them until they run too far (100 studs)

    💀 Despawn if they stray too far from home

    🛑 Stop chasing if players escape detection radius

In [ ]:
local npc = script.Parent
local humanoid = npc:WaitForChild("Humanoid")
local pathfindingService = game:GetService("PathfindingService")
local players = game:GetService("Players")
local runService = game:GetService("RunService")
local serverStorage = game:GetService("ServerStorage")

local detectionRadius = 50
local maxChaseDistance = 100
local followTarget = nil
local npcHomePosition = npc.HumanoidRootPart.Position
local isFollowing = false

-- Create a clone backup to respawn
local npcClone = npc:Clone()
npcClone.Parent = nil

-- Function to find closest player within range
local function getNearestPlayer()
	local closest = nil
	local closestDist = detectionRadius

	for _, player in ipairs(players:GetPlayers()) do
		if player.Character and player.Character:FindFirstChild("HumanoidRootPart") then
			local dist = (npc.HumanoidRootPart.Position - player.Character.HumanoidRootPart.Position).Magnitude
			if dist <= closestDist then
				closestDist = dist
				closest = player
			end
		end
	end

	return closest
end

-- Follow the target player using Pathfinding
local function followPlayer(player)
	if not player.Character then return end
	local hrp = player.Character:FindFirstChild("HumanoidRootPart")
	if not hrp then return end

	local path = pathfindingService:CreatePath({
		AgentRadius = 2,
		AgentHeight = 5,
		AgentCanJump = true,
		AgentJumpHeight = 10,
		AgentCanClimb = true,
	})

	path:ComputeAsync(npc.HumanoidRootPart.Position, hrp.Position)

	if path.Status == Enum.PathStatus.Success then
		for _, waypoint in pairs(path:GetWaypoints()) do
			if not followTarget then return end -- Exit if follow canceled
			if waypoint.Action == Enum.PathWaypointAction.Jump then
				humanoid:ChangeState(Enum.HumanoidStateType.Jumping)
			end
			humanoid:MoveTo(waypoint.Position)
			humanoid.MoveToFinished:Wait()
		end
	end
end

-- Main loop
runService.Heartbeat:Connect(function()
	if not followTarget then
		local found = getNearestPlayer()
		if found then
			followTarget = found
			isFollowing = true
		end
	else
		if followTarget.Character and followTarget.Character:FindFirstChild("HumanoidRootPart") then
			local playerDist = (npc.HumanoidRootPart.Position - followTarget.Character.HumanoidRootPart.Position).Magnitude
			local homeDist = (npc.HumanoidRootPart.Position - npcHomePosition).Magnitude

			-- Stop if player goes out of 50
			if playerDist > detectionRadius then
				followTarget = nil
				isFollowing = false
				return
			end

			-- Despawn if too far from home
			if homeDist > maxChaseDistance then
				local newNPC = npcClone:Clone()
				newNPC.Parent = workspace
				newNPC:SetPrimaryPartCFrame(CFrame.new(npcHomePosition))
				npc:Destroy()
				return
			end

			-- Continue following
			task.spawn(function()
				followPlayer(followTarget)
			end)
		else
			followTarget = nil
			isFollowing = false
		end
	end
end)
